In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration - If you have CUDA configured, you must use it. Try training with CPU and observe what happens
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#Setting a seed for torch
'''step - Your seed will be the last 6 digits of your A# excluding any leading zeros'''
torch.manual_seed(298742) 


In [2]:
#Check your Current Working Directory
!pwd

/home/jaren/development/school/deepLearning/hw3


In [3]:
#Set Batch Size
'''Step - Set the correct batch size. '''
batch_size = 32


# Download MNIST dataset to local drive. A new folder "data" will be created in the current directory to store data
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())
# Data loader for shuffling and batching.
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)



In [4]:
# Hyper-parameters - We are specifying these apriori
#Network Architecture
input_size = 784
'''Output layer has 10 nodes because we want to predict 10 classes'''
num_classes = 10 

#Training Parameters
''' - Define the number of epochs and observe the changes'''
num_epochs = 25



# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        '''
        Step - Define the N/w architecture. Use RELU Activation
        '''
        
        ''' Step - Define a Linear Unit with input size and hidden size''' 
        self.fc1 = nn.Linear(input_size, hidden_size)
        ''' Step - Define a RELU Activation unit'''
        self.relu = nn.ReLU()
        ''' Step - Define a Linear Unit with input size and output size (number of classes for MNIST)''' 
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        '''
        Step - Forward Propagate through the layers as defined above. Fill in params in place of ...
        '''
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out



# Define the Loss Function and optimizer
'''Step - Define a proper loss function'''
criterion = nn.MSELoss()



In [8]:
# Train the model
'''step - choose different learning rates and store them in a list and observe the changes '''
learning_rates = [.0007]
'''Vary this number and observe the changes, define a list of possible values'''
hidden_sizes = [512]

for hidden_size in hidden_sizes:
    for learning_rate in learning_rates:
        total_step = len(train_loader)
        #Define the Model Object and your optimizer
        model = NeuralNet(input_size, hidden_size, num_classes).to(device)
        '''Step - Invoke an appropriate optimizer that takes a ...?'''  
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
        for epoch in range(num_epochs):
            for i, (images, labels) in enumerate(train_loader):  
                # Move tensors to the configured device
                images = images.reshape(-1, 28*28).to(device)
                labels = labels.to(device)

                # Forward pass
                '''Step - Get Network outputs with forward propagation with current network weights'''
                outputs = model(images)
                '''Step - Get Loss by comparing outputs with True Labels after forward propagation'''

                transformed_labels = torch.zeros(batch_size, num_classes).to(device)

                for i, label in enumerate(labels):
                    transformed_labels[i][label] = 1

                loss = criterion(outputs, transformed_labels)



                # Backward and optimize

                '''Step - ... below needs to be replaced with functions'''
                '''Step - clear the gradients after each pass - Strongly recommended'''
                optimizer.zero_grad() 
                '''Backpropagate the Loss to calculate gradient for each weight'''
                loss.backward()
                '''Update the weight using the learning rate'''
                optimizer.step() 

                #Print Progress every 100 steps
                if (i+1) % 5 == 0:
                    print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                           .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
                    
    # Test the model once you finish training 

        with torch.no_grad(): # In test phase, we don't need to compute gradients (for memory efficiency)
            correct = 0
            total = 0
            for images, labels in test_loader:
                
                '''Step - Move images to device after appropriate reshaping'''
                images = images.reshape(-1, 28*28).to(device)

                '''Step  - Move labels to device'''
                labels = labels.to(device)

                #get network outputs
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

            print(f'Accuracy of the network on the 10000 test images: {(100 * correct / total)} %, with learning rate: {learning_rate}, and {hidden_size} hidden neurons')

Accuracy of the network on the 10000 test images: 98.21 %, with learning rate: 0.0007, and 512 hidden neurons


In [ ]:
# Save the model checkpoint for future use
torch.save(model.state_dict(), 'model.ckpt')